# このページについて
このページでは，研究・学習・教育を目的として，鈴木大慈先生の著作『[確率的最適化](https://www.kspub.co.jp/book/detail/1529076.html)』（講談社）に基づく内容の要約・補足・解説を行っています．  

# 注意
このページは著作権法第32条（引用）および第30条（私的使用のための複製）の趣旨に従い，原著の価値や意図を損なうことなく，個人の学習を目的として構成されています．

このページの内容は原著の代替ではなく，必ず原書の購読を推奨いたします．  
著者および出版社の知的財産権を尊重する立場から，不適切な記載や引用が含まれていた場合には，速やかに修正・削除等の対応をいたします．

万が一，著作権等の侵害に該当する箇所がある場合には，以下の GitHub の issue または pull request，あるいは Discussionsにてご連絡いただけますと幸いです．

# 確率的鏡像降下法
確率的勾配降下法では，正則化をする目的でパラメータの更新が保守的になるようにノルムなどを使っていました．一方でパラメータ空間でメトリックを図る方法として，ユークリッド距離が最適とは限りません．ここで，以下のようにユークリッド空間を拡張したブレぐマンダイバージェンスを定義します．

## ブレグマンダイバージェンス
$\phi: \mathbb{R}^p \rightarrow \mathbb{R}$：連続で微分可能な狭義凸関数とします．ブレグマンダイバージェンスは，

$$
B_\phi\left(\beta \| \beta^{\prime}\right):=\phi(\beta)-\phi\left(\beta^{\prime}\right)-\left\langle\nabla \phi\left(\beta^{\prime}\right), \beta-\beta^{\prime}\right\rangle
$$
と定義されます．

## ブレグマンダイバージェンスの可視化
定義域内の二つの点のブレグマンダイバージェンスは，以下のように可視化できます．基準とする点の接線を考え，もう一つの点の接線における値と，その点における関数の値の差分です．

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Layout
import ipywidgets as widgets

# --- Bregmanダイバージェンスの計算関数 (再掲) ---
def quadratic_function(x, a, b, c):
    return a * x**2 + b * x + c

def gradient_quadratic_function(x, a, b):
    return 2 * a * x + b

def bregman_divergence_quadratic(p_val, q_val, a_coeff, b_coeff, c_coeff):
    phi_p = quadratic_function(p_val, a_coeff, b_coeff, c_coeff)
    phi_q = quadratic_function(q_val, a_coeff, b_coeff, c_coeff)
    grad_phi_q = gradient_quadratic_function(q_val, a_coeff, b_coeff)
    dot_product = grad_phi_q * (p_val - q_val)
    divergence = phi_p - phi_q - dot_product
    # 厳密には a > 0 の場合のみ定義されるが、ここでは負の値も許容してプロット
    if a_coeff == 0: # 厳密には凸でないが、0除算を避ける
        return np.inf if p_val != q_val else 0
    return divergence

# --- プロット関数 ---
def plot_bregman_divergence_interactive(p_val, q_val, a_coeff):
    # bとcはダイバージェンスの計算に影響しないが、関数の形状のために固定
    b_coeff = 0.5
    c_coeff = 1.0

    if a_coeff <= 0:
        print("警告: Bregmanダイバージェンスの生成関数は厳密に凸である必要があります (a > 0)。")
        # プロットは試みるが、解釈に注意
        # return

    x_range = np.linspace(min(p_val, q_val) - 5, max(p_val, q_val) + 5, 200)
    phi_x = quadratic_function(x_range, a_coeff, b_coeff, c_coeff)

    phi_q_val = quadratic_function(q_val, a_coeff, b_coeff, c_coeff)
    grad_phi_q_val = gradient_quadratic_function(q_val, a_coeff, b_coeff)

    # qにおける接線: y = grad_phi(q) * (x - q) + phi(q)
    tangent_at_q = grad_phi_q_val * (x_range - q_val) + phi_q_val

    # Bregmanダイバージェンスの計算
    div = bregman_divergence_quadratic(p_val, q_val, a_coeff, b_coeff, c_coeff)

    plt.figure(figsize=(10, 7))

    # 1. phi(x) のプロット
    plt.plot(x_range, phi_x, label=f"$\phi(x) = {a_coeff:.2f}x^2 + {b_coeff:.2f}x + {c_coeff:.2f}$", color='blue')

    # 2. 点 q と phi(q)
    plt.scatter([q_val], [phi_q_val], color='red', s=100, zorder=5, label=f"$q = {q_val:.2f}, \phi(q) = {phi_q_val:.2f}$")

    # 3. 点 q における接線
    plt.plot(x_range, tangent_at_q, '--', color='green', label=f"Tangent at $q$: $y = \phi(q) + \\nabla\phi(q)(x-q)$")

    # 4. 点 p と phi(p)
    phi_p_val = quadratic_function(p_val, a_coeff, b_coeff, c_coeff)
    plt.scatter([p_val], [phi_p_val], color='purple', s=100, zorder=5, label=f"$p = {p_val:.2f}, \phi(p) = {phi_p_val:.2f}$")

    # 5. Bregmanダイバージェンス D_phi(p,q) の視覚化
    # phi(p) から接線上の p に対応する点までの垂直線
    tangent_value_at_p = grad_phi_q_val * (p_val - q_val) + phi_q_val
    plt.vlines(p_val, tangent_value_at_p, phi_p_val, colors='orange', linestyles='dotted',
               label=f"$D_\\phi(p,q) = \phi(p) - (\phi(q) + \\nabla\phi(q)(p-q))$ \n $= {div:.3f}$")
    # y軸の範囲を調整
    y_min = min(phi_x.min(), tangent_at_q.min()) - abs(div)*0.5 - 1
    y_max = max(phi_x.max(), tangent_at_q.max()) + abs(div)*0.5 + 1
    if np.isinf(y_min) or np.isinf(y_max) or np.isnan(y_min) or np.isnan(y_max): # a=0 の場合など
        pass
    else:
        plt.ylim(y_min, y_max)


    plt.title(f"Bregman Divergence $D_\\phi(p,q)$ for Quadratic Function\n $D_\\phi(p,q) = {a_coeff:.2f}(p-q)^2 = {div:.3f}$")
    plt.xlabel("x")
    plt.ylabel("$\phi(x)$ / Tangent value")
    plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1))
    plt.grid(True)
    plt.tight_layout(rect=[0, 0, 0.85, 1]) # 凡例スペース確保
    plt.show()

# --- インタラクティブウィジェットの作成 ---
p_slider = FloatSlider(min=-10.0, max=10.0, step=0.1, value=5.0, description='p:', continuous_update=False, layout=Layout(width='80%'))
q_slider = FloatSlider(min=-10.0, max=10.0, step=0.1, value=2.0, description='q:', continuous_update=False, layout=Layout(width='80%'))
a_slider = FloatSlider(min=0.1, max=5.0, step=0.1, value=0.5, description='a (coeff of $x^2$):', continuous_update=False, layout=Layout(width='80%'))
# a=0 の場合も許容するなら min=0.0 とするが、Bregmanダイバージェンスの定義上 a>0 が望ましい

# interact 関数でウィジェットとプロット関数を連携
interactive_plot = interact(
    plot_bregman_divergence_interactive,
    p_val=p_slider,
    q_val=q_slider,
    a_coeff=a_slider
)

# 初期プロット表示
# plot_bregman_divergence_interactive(p_slider.value, q_slider.value, a_slider.value)

interactive(children=(FloatSlider(value=5.0, continuous_update=False, description='p:', layout=Layout(width='8…

## アルゴリズム
$\beta_0 \in \mathcal{B}$を初期化
for$t=1,2, \ldots, T$：
	observe
		$z_t \sim P(Z)$
	compute
		$g_t \in \partial \ell_t\left(\beta_{t-1}\right)$
	update$\beta_t$
$$
\beta_t=\underset{\beta \in \mathcal{B}}{\operatorname{argmin}}\left\{\left\langle g_t, \beta\right\rangle+\psi(\beta)+\eta_t B_\phi\left(\beta \| \beta_{t-1}\right)\right\} .
$$
## 鏡像降下法の理解

普通の勾配降下法は，主変数であるパラメータの世界で最適化を行います．鏡像降下法は，直接パラメータ空間の鏡に映ったような別の空間である双対空間で最適化というか勾配降下を行います．そしてその結果を元のパラメータ空間に返します．

では，この鏡(双対空間への変換)は何を示しますでしょうか？それは狭義凸関数で連続微分可能な$\phi(\beta)$です．ポテンシャル関数とか言います．

主変数は$\beta$です．一方で双対変数は$\theta = \nabla\phi(\beta)$です．狭義凸関数という仮定をポテンシャル関数においているので，$\beta \leftrightarrow \nabla\phi(\beta)$は1対1対応になります．$\nabla\phi(\beta)$から元の$\beta$に戻るには，$\phi$の凸共役関数$\phi^*$の勾配を使います．具体的には，$\beta = \nabla\phi^*(\nabla\phi(\beta))$という関係が成り立ちます．
## 鏡像降下法の更新ステップの分解

$$
\beta_t=\underset{\beta \in \mathcal{B}}{\operatorname{argmin}}\left\{\left\langle g_t-\eta_t \nabla \phi\left(\beta_{t-1}\right), \beta\right\rangle+\eta_t \phi(\beta)\right\}
$$
$\beta$に関する勾配が0になるとき，上の式は最小化されます．つまり，
$$
g_t - \eta_t \nabla\phi(\beta_{t-1}) + \eta_t \nabla\phi(\beta_t) = 0
$$
となります．上の式を$\nabla\phi(\beta_t)$について解くと，
$$
\nabla\phi(\beta_t) = \nabla\phi(\beta_{t-1}) - \frac{1}{\eta_t} g_t
$$
となります．$\nabla\phi(\beta_{t-1})$は，1ステップ前の主変数$\beta_{t-1}$の双対変数です．$g_t$は現在の損失関数の$\beta_{t-1}$における勾配です．これも双対空間の要素とみなせます．損失関数の勾配は，主変数の微小変化に対する関数の変化率であり，双対的な性質を持つからです．つまり，現在の双対変数$\nabla\phi(\beta_t)$は，前の双対変数$\nabla\phi(\beta_{t-1})$から，現在の勾配$g_t$の方向（スケーリング付き）に更新されることを示しているわけです．

$\nabla\phi(\beta_t)$を獲得したら，これを元の主変数$\beta_t$に戻します．
$$
\beta_t = \nabla\phi^*\left(\nabla\phi(\beta_t)\right) = \nabla\phi^*\left(\nabla\phi(\beta_{t-1}) - \frac{1}{\eta_t} g_t\right)
$$
これが鏡像降下法の最終的な更新式です．つまり，
1.  主空間$\to$双対空間:$\beta_{t-1} \xrightarrow{\nabla\phi} \nabla\phi(\beta_{t-1})$（鏡映）
2.  双対空間で更新:$\nabla\phi(\beta_{t-1}) - \frac{1}{\eta_t} g_t$（鏡像）
3.  双対空間$\to$主空間:$\nabla\phi(\beta_{t-1}) - \frac{1}{\eta_t} g_t \xrightarrow{\nabla\phi^*} \beta_t$（鏡映）


## 鏡像降下法の収束レート
### 仮定1：ブレグマンダイバージェンスの下界

$$
B_\phi\left(\beta \| \beta^{\prime}\right) \geq \frac{1}{2}\left\|\beta-\beta^{\prime}\right\|_{H_\phi}^2 \quad \left(\forall \beta, \beta^{\prime} \in \mathcal{B}\right)
$$

* $H_\phi \succ O$:$H_\phi$はある正定値対称行列です

この仮定は，ポテンシャル関数$\phi$が，ある行列$H_\phi$に関して強く凸 であることを意味しています．通常の強凸性の定義は，関数$f(x)$に対して，ある$\mu > 0$が存在して，
$$
f(x) \ge f(y) + \langle \nabla f(y), x-y \rangle + \frac{\mu}{2} \|x-y\|^2
$$
が成り立つことです．これとブレグマンダイバージェンスの定義を比較すると，この仮定は，
$$
\phi(\beta) \ge \phi(\beta') + \langle \nabla\phi(\beta'), \beta-\beta' \rangle + \frac{1}{2} \|\beta-\beta'\|_{H_\phi}^2
$$
と同等です．つまり，$\phi$は点$\beta'$における接平面よりも，少なくとも$\frac{1}{2}\|\beta-\beta'\|_{H_\phi}^2$だけ上にあるということです．$H_\phi$が単位行列$I$のスカラー倍$\mu I$であれば，通常のユークリッドノルムに関する$\mu$-強凸性と同じになります．

### 仮定2：双対ノルムに関する勾配の期待値が有界

$$
\exists G>0 , \forall t  , \mathrm{E}\left[\left\|g_t\right\|_{H_\phi^{-1}}^2\right] \leq G^2
$$
### 定理

任意の$\beta^* \in \mathcal{B}$を固定します．仮定のもと$\mathrm{E}_{z_{1:t}}[B_\phi(\beta^* \| \beta_t)] \leq D^2/2 \quad (\forall t \geq 1)$，$\eta_t = \sqrt{t}\eta_0$とおけば，

$$
\mathrm{E}_{z_{1:T}}\left[L_\psi\left(\frac{1}{T+1} \sum_{t=0}^T \beta_t\right)\right]-L_\psi\left(\beta^*\right) \leq \frac{\eta_0 D^2+G^2 / \eta_0}{\sqrt{T+1}}
$$
### 証明

鏡像降下法の更新則とブレグマンダイバージェンスの性質（特に三点恒等式と，証明ヒントにある内積評価$\langle g_t, \cdot \rangle \le \frac{1}{2\eta_t}\|\cdot\|_{H_\phi^{-1}}^2 + \eta_t B_\phi(\cdot\|\cdot)$）を用いると，以下のような形の不等式が得られます．
$$
\lambda_t (\mathrm{E}[L_\psi(\beta_{t-1})] - L_\psi(\beta^*)) \le \mathrm{E}[B_\phi(\beta^* \| \beta_{t-1})] - \mathrm{E}[B_\phi(\beta^* \| \beta_t)] + \frac{\lambda_t^2}{2} G^2
$$
上記不等式を$t=0$から$T$まで足し合わせると，右辺の$\mathrm{E}[B_\phi(\beta^* \| \beta_t)]$の項は途中の項が打ち消し合う和を形成し，$\mathrm{E}[B_\phi(\beta^* \| \beta_0)]$と最後のダイバージェンスの差として評価できます．

定理で指定されている係数$\eta_t = \sqrt{t}\eta_0$を用いて，和の不等式を整理します．

最後に，$\sum (\mathrm{E}[L_\psi(\beta_t)] - L_\psi(\beta^*))$のような和を，Jensenの不等式を用いて$\mathrm{E}[L_\psi(\frac{1}{T+1}\sum \beta_t)] - L_\psi(\beta^*)$の形に変換します．
これにより，左辺の和にかかっていた$1/(T+1)$で両辺を割ることで，目的の$O(1/\sqrt{T+1})$の収束レート
$$\mathrm{E}\left[L_\psi\left(\frac{1}{T+1} \sum_{t=0}^T \beta_t\right)\right]-L_\psi\left(\beta^*\right) \leq \frac{\eta_0 D^2+G^2 / \eta_0}{\sqrt{T+1}}$$
が得られます．